<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-03 12:31:22
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.45 C
-------------------
Today PnL: 19.60 K (0.14%)
Current PnL: -19.64 L (-12.87%)
CY Booked + Current PnL: -6.99 L (-4.58%)
-------------------
Total profit:  2.29 L
Total loss:  -21.92 L
-------------------
Total Booked + Current PnL: 19.99 L (15.91%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.73%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.68 L (59.81%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.44%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,68.0,X-LC,5.29,232050.0,21684.0,7101.0,0.07,10.31,3.06,13.68,37.0,3.05,1.61,28.53,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,106.62,58.0,M-SC,1.53,89060.0,-11717.0,11809.0,1.50,-11.63,13.26,0.09,245.0,-0.99,0.62,17.01,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.44,170902.0,30062.0,14424.0,1.10,21.34,8.44,31.59,66.0,2.08,1.19,53.49,XY24,NTT,BANKS
33,ICICIGI,2252.93,-13.84,59.0,X-MC,5.04,148200.0,12067.0,18510.0,0.45,8.86,12.49,22.47,91.0,0.65,1.03,22.99,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-11.87,31.0,X-MC,2.38,87045.0,-3633.0,18541.0,0.07,-4.01,21.30,16.44,101.0,-0.20,0.61,15.36,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,-15.09,64.0,H-SC,8.80,96555.0,-1425.0,23028.0,5.37,-1.45,23.85,22.05,152.0,-0.06,0.67,40.35,XR,ATH,FINANCE
59,REPCOHOME,880.00,-53.06,73.0,H-SC,6.05,268923.0,-17582.0,274920.0,3.90,-6.14,102.23,89.82,134.0,-0.06,1.87,42.28,XY24,NTT,FINANCE
81,VAIBHAVGBL,521.00,81.82,75.0,H-SC,1.74,157723.0,-25052.0,136115.0,3.89,-13.71,86.30,60.77,125.0,-0.18,1.10,41.27,XR,NTT,JEWELLERY
64,SFL,1287.00,41.13,68.0,M-SC,5.16,186328.0,-75890.0,145727.0,3.27,-28.94,78.21,26.63,229.0,-0.52,1.30,15.85,XY24,NTT,MISC
70,SURYODAY,240.00,73.25,64.0,H-SC,15.65,159824.0,-19247.0,84978.0,2.76,-10.75,53.17,36.71,135.0,-0.23,1.11,58.58,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VOLTAS,1530.0,-2.90,39.0,X-MC,4.56,203625.0,11883.0,25881.0,-1.89,6.20,12.71,19.69,99.0,0.46,1.42,13.42,XY25,NTT,AC
9,AWL,485.0,-63.83,56.0,X-MC,1.75,257898.0,-43853.0,205287.0,-1.75,-14.53,79.60,53.50,116.0,-0.21,1.79,14.55,XY24,NTT,FMCG
41,ITC,452.0,-37.50,53.0,X-LC,1.19,202446.0,2308.0,18585.0,-1.51,1.15,9.18,10.44,4.0,0.12,1.41,6.78,X40,NTT,FMCG
85,WHIRLPOOL,2270.0,-38.90,59.0,M-SC,4.25,126794.0,11996.0,82048.0,-1.49,10.45,64.71,81.92,223.0,0.15,0.88,50.10,XR,NTT,DURABLES
53,QUESS,986.0,-47.66,41.0,X-SC,37.78,51774.0,-13232.0,164160.0,-1.39,-20.35,317.07,232.18,198.0,-0.08,0.36,0.99,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,106.62,58.0,M-SC,1.53,89060.0,-11717.0,11809.0,1.50,-11.63,13.26,0.09,245.0,-0.99,0.62,17.01,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,46.0,H-SC,2.69,222309.0,-47358.0,83699.0,-0.38,-17.56,37.65,13.47,138.0,-0.57,1.55,13.04,XY24,NTT,PAINTS
17,CERA,9475.0,-22.43,52.0,H-SC,2.17,141473.0,-34430.0,76452.0,-1.21,-19.57,54.04,23.89,149.0,-0.45,0.98,22.18,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-24.05,45.0,H-MC,6.12,102686.0,-29149.0,71377.0,-0.92,-22.11,69.51,32.03,98.0,-0.41,0.71,15.02,OX40N,NTT,IT
67,SIS,528.0,2009.33,42.0,H-SC,3.28,85119.0,-25913.0,49522.0,0.15,-23.34,58.18,21.26,156.0,-0.52,0.59,15.00,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,66.0,M-LC,10.44,170902.0,30062.0,14424.0,1.1,21.34,8.44,31.59,66.0,2.08,1.19,53.49,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,52.37,63.0,M-MC,9.59,240167.0,15205.0,152458.0,1.91,6.76,63.48,74.53,192.0,0.10,1.67,41.19,XY24,BTT,STEEL
56,RECLTD,446.00,47.87,53.0,M-LC,5.20,206938.0,3768.0,38366.0,0.37,1.85,18.54,20.74,55.0,0.10,1.44,8.84,XY25,NTT,FINANCE
31,HINDZINC,730.22,28.52,50.0,M-LC,10.89,209104.0,4028.0,108546.0,0.88,1.96,51.91,54.89,52.0,0.04,1.45,26.53,X5K,ATH,METALS
46,KPIGREEN,731.05,13.63,76.0,H-SC,5.74,135324.0,10027.0,48906.0,0.32,8.00,36.14,47.03,141.0,0.21,0.94,69.44,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.44,170902.0,30062.0,14424.0,1.10,21.34,8.44,31.59,66.0,2.08,1.19,53.49,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,13.63,76.0,H-SC,5.74,135324.0,10027.0,48906.0,0.32,8.00,36.14,47.03,141.0,0.21,0.94,69.44,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-38.90,59.0,M-SC,4.25,126794.0,11996.0,82048.0,-1.49,10.45,64.71,81.92,223.0,0.15,0.88,50.10,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.33,130709.0,7373.0,124252.0,0.34,5.98,95.06,106.72,119.0,0.06,0.91,30.75,AR,ATH,MISC
31,HINDZINC,730.22,28.52,50.0,M-LC,10.89,209104.0,4028.0,108546.0,0.88,1.96,51.91,54.89,52.0,0.04,1.45,26.53,X5K,ATH,METALS
86,WIPRO,420.00,-15.15,42.0,M-LC,5.34,148856.0,-2089.0,111538.0,-0.24,-1.38,74.93,72.51,53.0,-0.02,1.03,4.56,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,19.0,X-LC,24.25,148830.0,-127693.0,237771.0,-0.59,-46.18,159.76,39.81,54.0,-0.54,1.03,4.90,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.78,88901.0,-39769.0,85069.0,0.05,-30.91,95.69,35.20,219.0,-0.47,0.62,2.04,X40,NTT,FOOTWEAR
23,DMART,5391.45,-19.19,29.0,X-LC,8.96,206005.0,-4402.0,63573.0,-0.80,-2.09,30.86,28.12,38.0,-0.07,1.43,21.04,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,-11.87,31.0,X-MC,2.38,87045.0,-3633.0,18541.0,0.07,-4.01,21.30,16.44,101.0,-0.20,0.61,15.36,X40,ATH,PHARMA
44,JSWINFRA,342.00,-22.53,31.0,X-MC,4.95,194434.0,-5913.0,36417.0,-0.62,-2.95,18.73,15.23,178.0,-0.16,1.35,28.35,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-37.50,53.0,X-LC,1.19,202446.0,2308.0,18585.0,-1.51,1.15,9.18,10.44,4.0,0.12,1.41,6.78,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.04,47.0,H-LC,1.25,155965.0,-30130.0,77608.0,0.74,-16.19,49.76,25.51,15.0,-0.39,1.08,14.93,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-30.55,39.0,X-MC,1.65,200925.0,105.0,67692.0,0.33,0.05,33.69,33.76,80.0,0.00,1.40,5.25,X40,ATH,FMCG
28,HAVELLS,2069.16,-6.04,47.0,X-MC,2.88,313656.0,-14335.0,120883.0,-0.02,-4.37,38.54,32.48,92.0,-0.12,2.18,6.56,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-5.67,73.0,X-LC,4.36,247216.0,13072.0,35550.0,0.41,5.58,14.38,20.77,86.0,0.37,1.72,12.89,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,41.0,X-SC,37.78,51774.0,-13232.0,164160.0,-1.39,-20.35,317.07,232.18,198.0,-0.08,0.36,0.99,XY24,NTT,MISC
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.78,88901.0,-39769.0,85069.0,0.05,-30.91,95.69,35.20,219.0,-0.47,0.62,2.04,X40,NTT,FOOTWEAR
20,COLPAL,3726.84,-5.86,44.0,X-MC,7.41,219590.0,-43775.0,153098.0,-1.04,-16.62,69.72,41.51,84.0,-0.29,1.53,3.05,XY25,ATH,FMCG
51,PAGEIND,51769.93,-27.90,40.0,X-MC,6.92,81460.0,-2050.0,22084.0,-1.14,-2.45,27.11,23.98,82.0,-0.09,0.57,3.22,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-14.48,47.0,X-MC,6.00,180318.0,-20188.0,117351.0,-0.24,-10.07,65.08,48.45,90.0,-0.17,1.25,3.98,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,41.0,X-SC,37.78,51774.0,-13232.0,164160.0,-1.39,-20.35,317.07,232.18,198.0,-0.08,0.36,0.99,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.90,40.0,X-MC,6.92,81460.0,-2050.0,22084.0,-1.14,-2.45,27.11,23.98,82.0,-0.09,0.57,3.22,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.17,42.0,X-SC,4.58,81545.0,-63615.0,139540.0,-0.55,-43.82,171.12,52.31,136.0,-0.46,0.57,9.15,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.87,31.0,X-MC,2.38,87045.0,-3633.0,18541.0,0.07,-4.01,21.30,16.44,101.0,-0.20,0.61,15.36,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.78,88901.0,-39769.0,85069.0,0.05,-30.91,95.69,35.20,219.0,-0.47,0.62,2.04,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,49.0,X-LC,10.26,283955.0,-62001.0,128688.0,-1.22,-17.92,45.32,19.28,1.0,-0.48,1.97,4.97,X40,ATH,IT
39,INFY,2275.00,-19.24,51.0,X-LC,5.38,314814.0,1926.0,169748.0,-0.29,0.62,53.92,54.87,3.0,0.01,2.19,8.95,X40,BTT,IT
41,ITC,452.00,-37.50,53.0,X-LC,1.19,202446.0,2308.0,18585.0,-1.51,1.15,9.18,10.44,4.0,0.12,1.41,6.78,X40,NTT,FMCG
83,VBL,671.64,-15.98,55.0,X-LC,7.53,302540.0,-13302.0,125978.0,0.97,-4.21,41.64,35.67,5.0,-0.11,2.10,9.06,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,51.0,H-LC,5.34,250800.0,-10819.0,130040.0,0.10,-4.14,51.85,45.57,7.0,-0.08,1.74,6.72,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7083.33,60.0,L-SC,8.76,78820.0,-14990.0,91605.0,-0.14,-15.98,116.22,81.67,269.0,-0.16,0.55,52.64,XR,NTT,CERAMICS
19,COFFEEDAY,80.00,-45.67,45.0,L-SC,28.41,79795.0,-33754.0,73802.0,-0.74,-29.73,92.49,35.27,268.0,-0.46,0.55,95.30,XR,NTT,HOTELS
49,MASFIN,398.61,-15.09,64.0,H-SC,8.80,96555.0,-1425.0,23028.0,5.37,-1.45,23.85,22.05,152.0,-0.06,0.67,40.35,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-38.90,59.0,M-SC,4.25,126794.0,11996.0,82048.0,-1.49,10.45,64.71,81.92,223.0,0.15,0.88,50.10,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.33,130709.0,7373.0,124252.0,0.34,5.98,95.06,106.72,119.0,0.06,0.91,30.75,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,81.82,75.0,H-SC,1.74,157723.0,-25052.0,136115.0,3.89,-13.71,86.30,60.77,125.0,-0.18,1.10,41.27,XR,NTT,JEWELLERY
46,KPIGREEN,731.05,13.63,76.0,H-SC,5.74,135324.0,10027.0,48906.0,0.32,8.00,36.14,47.03,141.0,0.21,0.94,69.44,MH,ATH,POWER
59,REPCOHOME,880.00,-53.06,73.0,H-SC,6.05,268923.0,-17582.0,274920.0,3.90,-6.14,102.23,89.82,134.0,-0.06,1.87,42.28,XY24,NTT,FINANCE
85,WHIRLPOOL,2270.00,-38.90,59.0,M-SC,4.25,126794.0,11996.0,82048.0,-1.49,10.45,64.71,81.92,223.0,0.15,0.88,50.10,XR,NTT,DURABLES
5,ANGELONE,3033.00,17.86,66.0,X-SC,5.84,208674.0,17668.0,40024.0,2.10,9.25,19.18,30.21,157.0,0.44,1.45,32.06,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,44.04
2,50,75.49


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,44.22
LC,32.96
MC,22.86


In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,30.00
X40,16.13
X40N,12.43
XY25,11.94
XR,9.98
AR,7.80
OX40N,6.52
X200,1.78
X5K,1.45


In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
H-SC,24.51
X-LC,22.25
X-MC,18.53
M-SC,12.72
X-SC,5.56
M-LC,5.11
H-LC,4.60
H-MC,2.34
M-MC,1.67


In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.05,-4.53,38.32
IT,12.40,-17.75,80.90
FINANCE,12.26,-7.21,55.81
MISC,7.84,-15.41,73.62
ELECTRICAL,5.77,-8.78,47.35
PAINTS,5.45,-17.93,35.51
BANKS,4.09,-14.89,81.72
INSURANCE,3.83,-0.47,37.94
AUTO,2.75,-45.15,105.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3259279.0,22
XR,1221014.0,13
AR,1165436.0,9
X40,839236.0,12
X40N,680606.0,9
OX40N,546409.0,9
XY25,462696.0,8
SR,267809.0,2
X5K,108546.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3307423.0,24
M-SC,1217526.0,15
X-MC,1210456.0,14
X-LC,1077712.0,13
X-SC,550966.0,6
H-LC,276065.0,3
M-LC,272874.0,4
L-SC,254933.0,3
H-MC,250508.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1156959.0      6
           AR         833614.0      5
           XR         757677.0      7
M-SC       XY24       698625.0      6
X-MC       XY24       586093.0      4
X-LC       X40        462551.0      5
           XY24       307429.0      2
X-SC       X40N       301737.0      4
X-MC       X40        291616.0      6
H-SC       SR         267809.0      2
           OX40N      242458.0      3
X-LC       X40N       225101.0      3
H-LC       AR         207648.0      2
H-MC       XY24       179131.0      1
X-MC       XY25       178979.0      2
L-SC       XR         165407.0      2
X-SC       XY24       164160.0      1
X-MC       X40N       153768.0      2
M-MC       XY24       152458.0      1
M-SC       OX40N      143048.0      4
           XR         129377.0      2
           AR         124174.0      2
           XY25       122302.0      1
M-LC       XR         111538.0      1
           X5K        108546.0      1
L-SC       OX40N       89526.0      1
X-SC       X40         85069.0      1
X-LC       XY25        82631.0      3
H-MC       OX40N       71377.0      1
H-LC       X200        68417.0      1
L-MC       XR          57015.0      1
H-SC       MH          48906.0      1
L-LC       XY25        40418.0      1
M-LC       XY25        38366.0      1
           XY24        14424.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
